In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from pprint import pprint
from itertools import chain

from utils.data_loading import load_data
from utils.data_saving import save_processed_dataset
from utils.add_feature import add_row_wise_feature

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [2]:
RAW_DATA_DIR = "./data/input_data/data.csv"

contracts_df = load_data(RAW_DATA_DIR)

Total number of rows loaded: 1000
DF columns:
 id                           int32
application_date    datetime64[ns]
contracts                   object
dtype: object


In [3]:
FEATURES_DESCRIPTION_DIR = "./data/input_data/features.xlsx"

features_desc_df = pd.read_excel(FEATURES_DESCRIPTION_DIR)
features_desc_df

,No,Feature,Logic,If missing value
0,1,tot_claim_cnt_l180d,"Description: number of claims for last 180 days\nSource: contracts\nKey fields: claim_id, claim_date\nSpecial notes: \n1. In case claim date is null, don't take into consideration such claims.","In case no claims, then put -3 as a value of this feature."
1,2,disb_bank_loan_wo_tbc,"Description: Sum of exposue of loans without TBC loans. Exposure means here field ""loan_summa"".\nSource: contracts\nKey fields: bank, loan_summa, contract_date\nSpecial notes: \n1. Consider only loans where field ""bank"" is not in ['LIZ', 'LOM', 'MKO', 'SUG', null].\n2. Disbursed loans means loans where contract_date is not null","In case no claims, then put -3 as a value of this feature.\nIf no loans at all, then put -1 as a value of this feature."
2,3,day_sinlastloan,"Description: Number of days since last loan. \nSource: contracts\nKey fields: contract_date, summa\nSpecial notes:\n1. Take last loan of client where summa is not null and calculate number of days from contract_date of this loan to application date.","In case no claims at all, then put -3 as a value of this feature.\nIn case no loans at all, then put -1 as a value of this feature."


# Feature 1: Claims for last 180 days

In [4]:
def calculate_tot_claim_cnt_l180_per_id(
        contract_list: list[dict] | None,
        current_date: datetime.datetime
) -> int:
    """
    Description: number of claims for last 180 days
    Source: contracts
    Key fields: claim_id, claim_date

    Special notes:
    1. In case claim date is null, don't take into consideration such claims.

    :param contract_list: List of claim JSONs for current id.
    :param current_date: Last date, i.e. end of desired 180 days period.
    :return: Number of claims with `claim_date` not earlier than 180 days before `current_date`.
             -3 in case, when no such claims were found.
    """
    if contract_list is None:
        return -3

    claims_l180 = set()
    for contract in contract_list:
        if (
                contract['claim_date'] is not None and
                contract['claim_id'] not in claims_l180 and
                (current_date - contract['claim_date']).days < 180
        ):
            claims_l180.add(contract['claim_id'])

    if not claims_l180:
        return -3

    return len(claims_l180)

In [5]:
contracts_df = add_row_wise_feature(
    contracts_df,
    'tot_claim_cnt_l180',
    'contracts',
    calculate_tot_claim_cnt_l180_per_id,
    datetime.datetime.now()
)

In [6]:
contracts_df.tail(10)

,id,application_date,contracts,tot_claim_cnt_l180
990,2926200,2024-02-13 06:09:31.020,None,-3
991,2926201,2024-02-13 06:09:26.994,"[{'contract_id': 18410, 'bank': '004', 'summa': 49164700, 'loan_summa': 0, 'claim_date': 2019-04-03 00:00:00, 'claim_id': 18410, 'contract_date': 2019-04-03 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2021-02-22 00:00:00, 'claim_id': 43136, 'contract_date': None}, {'contract_id': None, 'bank': '062', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304534, 'contract_date': None}, {'contract_id': 3304674, 'bank': '062', 'summa': 405000000, 'loan_summa': 217320883, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304674, 'contract_date': 2022-02-14 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-18 00:00:00, 'claim_id': 62750, 'contract_date': None}, {'contract_id': None, 'bank': '014', 'summa': None, 'loan_summa': None, 'claim_date': 2023-05-31 00:00:00, 'claim_id': 'F00895453', 'contract_date': None}]",-3
992,2926202,2024-02-13 06:09:42.309,None,-3
993,2926203,2024-02-13 06:09:42.929,None,-3
994,2926204,2024-02-13 06:09:48.104,None,-3
995,2926205,2024-02-13 06:09:54.210,"[{'contract_id': 18410, 'bank': '004', 'summa': 49164700, 'loan_summa': 0, 'claim_date': 2019-04-03 00:00:00, 'claim_id': 18410, 'contract_date': 2019-04-03 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2021-02-22 00:00:00, 'claim_id': 43136, 'contract_date': None}, {'contract_id': None, 'bank': '062', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304534, 'contract_date': None}, {'contract_id': 3304674, 'bank': '062', 'summa': 405000000, 'loan_summa': 217320883, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304674, 'contract_date': 2022-02-14 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-18 00:00:00, 'claim_id': 62750, 'contract_date': None}, {'contract_id': None, 'bank': '014', 'summa': None, 'loan_summa': None, 'claim_date': 2023-05-31 00:00:00, 'claim_id': 'F00895453', 'contract_date': None}]",-3
996,2926206,2024-02-13 06:09:54.306,None,-3
997,2926207,2024-02-13 06:09:55.661,None,-3
998,2926208,2024-02-13 06:09:57.024,None,-3
999,2926209,2024-02-13 06:10:00.403,None,-3


In [7]:
contracts_df['tot_claim_cnt_l180'].unique()

array([-3], dtype=int64)

There is no issue, since the latest `claim_date` was `Timestamp('2024-02-13 00:00:00')`, and therefore all claims were performed more than 180 days ago.

# Feature 2: Exposue loans sum without TBC

In [8]:
def calculate_disb_bank_loan_wo_tbc_per_id(
        contracts_list: list[dict] | None
) -> float:
    """
    Description: Sum of exposue of loans without TBC loans. Exposure means here field ""loan_summa"".
    Source: contracts
    Key fields: bank, loan_summa, contract_date

    Special notes:
    1. Consider only loans where field ""bank"" is not in ['LIZ', 'LOM', 'MKO', 'SUG', null].
    2. Disbursed loans means loans where contract_date is not null"

    :param contracts_list: List of claim JSONs for current id.
    :return: Total sum of `loan_summa` of claims without TBC.
             -3 in case of no claims at all.
             -1 in case of no filled `loan_summa` for non-TBC claims were provided.
    """
    if not contracts_list:
        return -3

    not_tbc_claim_banks = ['LIZ', 'LOM', 'MKO', 'SUG', None]
    total_exposure_sum_not_tbc = 0.0
    total_exposure_num_not_tbc = 0

    for contract in contracts_list:
        if (
                'bank' in contract.keys() and
                contract['contract_date'] is not None and
                contract['loan_summa'] is not None and
                contract['bank'] not in not_tbc_claim_banks
        ):
            total_exposure_sum_not_tbc += contract['loan_summa']
            total_exposure_num_not_tbc += 1

    if total_exposure_num_not_tbc == 0:
        return -1

    return total_exposure_sum_not_tbc

In [9]:
contracts_df = add_row_wise_feature(
    contracts_df,
    'disb_bank_loan_wo_tbc',
    'contracts',
    calculate_disb_bank_loan_wo_tbc_per_id
)

In [10]:
contracts_df.tail(10)

,id,application_date,contracts,tot_claim_cnt_l180,disb_bank_loan_wo_tbc
990,2926200,2024-02-13 06:09:31.020,None,-3,-3.0
991,2926201,2024-02-13 06:09:26.994,"[{'contract_id': 18410, 'bank': '004', 'summa': 49164700, 'loan_summa': 0, 'claim_date': 2019-04-03 00:00:00, 'claim_id': 18410, 'contract_date': 2019-04-03 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2021-02-22 00:00:00, 'claim_id': 43136, 'contract_date': None}, {'contract_id': None, 'bank': '062', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304534, 'contract_date': None}, {'contract_id': 3304674, 'bank': '062', 'summa': 405000000, 'loan_summa': 217320883, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304674, 'contract_date': 2022-02-14 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-18 00:00:00, 'claim_id': 62750, 'contract_date': None}, {'contract_id': None, 'bank': '014', 'summa': None, 'loan_summa': None, 'claim_date': 2023-05-31 00:00:00, 'claim_id': 'F00895453', 'contract_date': None}]",-3,217320883.0
992,2926202,2024-02-13 06:09:42.309,None,-3,-3.0
993,2926203,2024-02-13 06:09:42.929,None,-3,-3.0
994,2926204,2024-02-13 06:09:48.104,None,-3,-3.0
995,2926205,2024-02-13 06:09:54.210,"[{'contract_id': 18410, 'bank': '004', 'summa': 49164700, 'loan_summa': 0, 'claim_date': 2019-04-03 00:00:00, 'claim_id': 18410, 'contract_date': 2019-04-03 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2021-02-22 00:00:00, 'claim_id': 43136, 'contract_date': None}, {'contract_id': None, 'bank': '062', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304534, 'contract_date': None}, {'contract_id': 3304674, 'bank': '062', 'summa': 405000000, 'loan_summa': 217320883, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304674, 'contract_date': 2022-02-14 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-18 00:00:00, 'claim_id': 62750, 'contract_date': None}, {'contract_id': None, 'bank': '014', 'summa': None, 'loan_summa': None, 'claim_date': 2023-05-31 00:00:00, 'claim_id': 'F00895453', 'contract_date': None}]",-3,217320883.0
996,2926206,2024-02-13 06:09:54.306,None,-3,-3.0
997,2926207,2024-02-13 06:09:55.661,None,-3,-3.0
998,2926208,2024-02-13 06:09:57.024,None,-3,-3.0
999,2926209,2024-02-13 06:10:00.403,None,-3,-3.0


In [11]:
contracts_df['disb_bank_loan_wo_tbc'].unique()

array([-3.00000000e+00,  0.00000000e+00, -1.00000000e+00,  2.53785175e+09,
        5.59300000e+08,  2.17682729e+09,  9.90510670e+09,  1.42166291e+09,
        2.33087600e+08,  3.00000000e+08,  1.93849380e+10,  1.03084840e+10,
        3.22525937e+09,  2.83999880e+10,  3.89592220e+08,  8.06595014e+09,
        7.00000000e+08,  4.34839653e+09,  8.15007727e+09,  1.00000000e+09,
        3.07403141e+08,  3.76755060e+09,  1.37499970e+09,  4.31182433e+08,
        7.04537061e+09,  3.69241481e+09,  4.16674000e+07,  1.36001300e+08,
        1.00800000e+09,  3.64000000e+08,  1.15287750e+10,  4.56374022e+08,
        1.10132494e+10,  3.33333500e+08,  5.91134623e+08,  3.36937499e+09,
        1.94764009e+09,  1.16264187e+09,  1.36127022e+09,  2.05400000e+08,
        4.49131404e+09,  1.46999880e+09,  7.07319423e+08,  1.12131700e+06,
        3.10220425e+10,  9.53687751e+08,  4.26275198e+08,  1.14877868e+09,
        2.50000000e+08,  3.27000000e+10,  1.48348581e+09,  3.28194520e+09,
        4.91809096e+08,  

# Feature 3: Number of days since last loan

In [12]:
def calculate_day_sinlastloan_per_id(
        contract_list: list[dict] | None,
        application_date: datetime.datetime
) -> int | None:
    """
    Description: Number of days since last loan.
    Source: contracts
    Key fields: contract_date, summa

    Special notes:
    1. Take last loan of client where summa is not null and calculate number of
       days from contract_date of this loan to application date.

    :param contract_list: List of claim JSONs for current id.
    :param application_date: Application date for the current id.
    :return: Number of days between last loan and application date.
             -3 in case of no claims at all.
             -1 in case of no filled `summa` were provided.
    """

    if not contract_list:
        return -3

    last_claim_date = datetime.datetime.min
    for contract in contract_list:
        if (
                contract['summa'] is not None and
                contract['claim_date'] is not None
        ):
            last_claim_date = max(last_claim_date, contract['claim_date'])

    if last_claim_date == datetime.datetime.min:
        return -1

    return (application_date - last_claim_date).days

In [13]:
contracts_df = add_row_wise_feature(
    contracts_df,
    'day_sinlastloan',
    'contracts',
    calculate_day_sinlastloan_per_id,
    contracts_df['application_date'].iloc[0],
    axis=1
)

In [14]:
contracts_df.tail(10)

,id,application_date,contracts,tot_claim_cnt_l180,disb_bank_loan_wo_tbc,day_sinlastloan
990,2926200,2024-02-13 06:09:31.020,None,-3,-3.0,-3
991,2926201,2024-02-13 06:09:26.994,"[{'contract_id': 18410, 'bank': '004', 'summa': 49164700, 'loan_summa': 0, 'claim_date': 2019-04-03 00:00:00, 'claim_id': 18410, 'contract_date': 2019-04-03 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2021-02-22 00:00:00, 'claim_id': 43136, 'contract_date': None}, {'contract_id': None, 'bank': '062', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304534, 'contract_date': None}, {'contract_id': 3304674, 'bank': '062', 'summa': 405000000, 'loan_summa': 217320883, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304674, 'contract_date': 2022-02-14 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-18 00:00:00, 'claim_id': 62750, 'contract_date': None}, {'contract_id': None, 'bank': '014', 'summa': None, 'loan_summa': None, 'claim_date': 2023-05-31 00:00:00, 'claim_id': 'F00895453', 'contract_date': None}]",-3,217320883.0,728
992,2926202,2024-02-13 06:09:42.309,None,-3,-3.0,-3
993,2926203,2024-02-13 06:09:42.929,None,-3,-3.0,-3
994,2926204,2024-02-13 06:09:48.104,None,-3,-3.0,-3
995,2926205,2024-02-13 06:09:54.210,"[{'contract_id': 18410, 'bank': '004', 'summa': 49164700, 'loan_summa': 0, 'claim_date': 2019-04-03 00:00:00, 'claim_id': 18410, 'contract_date': 2019-04-03 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2021-02-22 00:00:00, 'claim_id': 43136, 'contract_date': None}, {'contract_id': None, 'bank': '062', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304534, 'contract_date': None}, {'contract_id': 3304674, 'bank': '062', 'summa': 405000000, 'loan_summa': 217320883, 'claim_date': 2022-02-14 00:00:00, 'claim_id': 3304674, 'contract_date': 2022-02-14 00:00:00}, {'contract_id': None, 'bank': '004', 'summa': None, 'loan_summa': None, 'claim_date': 2022-02-18 00:00:00, 'claim_id': 62750, 'contract_date': None}, {'contract_id': None, 'bank': '014', 'summa': None, 'loan_summa': None, 'claim_date': 2023-05-31 00:00:00, 'claim_id': 'F00895453', 'contract_date': None}]",-3,217320883.0,728
996,2926206,2024-02-13 06:09:54.306,None,-3,-3.0,-3
997,2926207,2024-02-13 06:09:55.661,None,-3,-3.0,-3
998,2926208,2024-02-13 06:09:57.024,None,-3,-3.0,-3
999,2926209,2024-02-13 06:10:00.403,None,-3,-3.0,-3


In [16]:
contracts_df['day_sinlastloan'].unique()

array([  -3,  430,  288,  169,   -1,  157,   37,   68,   14,  144,   18,
       1729,  725,  434,  209,   21,  172,  992,  887,  470,  125,  699,
          5,  158,  283,    3,  124,  186,  369,  155,  298,  552,  105,
        679, 1361,   30,  112,  308, 2498,   40,   73,  933,  185,   53,
        235,   12,  176,   49,   19,   81,  356,   26,    8,  422,  350,
         55,   17,  222, 1699,   50,   69,  806,   91,  138,   83,   27,
          0, 1062,  296, 1329,  121, 1357,  150,  194,  117,   70,   87,
        173,  991,   94, 1025,  377,  146,   67,    7,   47, 1061, 1556,
        171,  266,  256,  145,  816, 1358, 1638,  154,   48,  728],
      dtype=int64)

# Saving dataset with additional features

In [17]:
PROCESSED_DATA_DIR = "./data/output_data/data_with_additional_features.csv"

save_processed_dataset(contracts_df, PROCESSED_DATA_DIR)